# Исследование надёжности заёмщиков

### Цели и задачи проекта

#Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### План работы

#### 1. Изучить входные данные;
#### 2. Выполнить предобработку данных;
#### 3. Ответить на поставленные Заказчиком вопросы.

### Информация о входных данных

В таблице двенадцать столбцов:<br>

**children** — количество детей в семье;<br>
**days_employed** — общий трудовой стаж в днях;<br>
**dob_years** — возраст клиента в годах;<br>
**education** — уровень образования клиента;<br>
**education_id** — идентификатор уровня образования;<br>
**family_status** — семейное положение;<br>
**family_status_id** — идентификатор семейного положения;<br>
**gender** — пол клиента;<br>
**income_type** — тип занятости;<br>
**debt** — имел ли задолженность по возврату кредитов;<br>
**total_income** — ежемесячный доход;<br>
**purpose** — цель получения кредита.<br>

### Оглавление

* [1. Открываем файл с данными и изучаем общую информацию](#first)
* [2. Предобработка данных](#second)
*  [Обработка пропусков](#third)
*    [Замена типа данных](#fourth)
*     [Обработка дубликатов](#fifth)
*     [Лемматизация](#sixth)
*     [Категоризация данных](#seventh)
* [3. Отвечаем на вопросы](#eighth)
*     [Есть ли зависимость между наличием детей и возвратом кредита в срок?](#ninth)
*     [Есть ли зависимость между семейным положением и возвратом кредита в срок?](#tenth)
*    [Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#eleventh)
*     [Как разные цели кредита влияют на его возврат в срок?](#twelfth)
* [4. Общий вывод](#thirteenth)

## 1. Открываем файл с данными и изучаем общую информацию. <a class="anchor" id="first"></a>

In [ ]:
import pandas as pd
from IPython.display import display

In [ ]:
clients = pd.read_csv('datasets/1_data.csv')
clients.info()
display(clients.head(10))

## Вывод

Получена таблица - статистика о платежеспособности проекта.

Каждая строчка в таблице соответствует информации о платежеспособности клиента. Всего 21525 строк.

В столбцах days_employed и total_income наблюдаем 2174 нулевых значений. Нулевые значения в столбцах трудовой стаж и<br>
доход не могут быть специальными, вероятно это ошибка ввода или подсчета, поэтому в следующей главе мы их обработаем.

В столбце days_employed обнаружен артефакт - отрицательное количество дней трудового стажа. Полагаю, что данный <br>
артефакт ввиду того, что дата начала работы и дата окончания работы были перепутаны местами и результат вычисления<br>
разности дат получился отрицательным. В шаге предобработка данных исправим этот артефакт.<br>

## 2. Предобработка данных <a class="anchor" id="second"></a>

### Обработка пропусков <a class="anchor" id="third"></a>

Рассмотрим природу появления нулевых значений в общем трудовом стаже и ежемесячном доходе для этого отсортируем <br>
строки по столбцу с трудовым стажем и выведем последние 15 значений: 

In [ ]:
display(clients.sort_values(by = 'days_employed', ascending = False).tail(15))

Как можем увидеть значения NaN имеется у клиентов, чей возраст гораздо больше 18 лет, с образованием и с различным<br>
типом занятости. Следовательно, отсутствовать трудовой стаж и ежемясячный доход у таких клиентов не может. <br>
Вероятно, NaN появился вследствие неправильного внесения данных. Поэтому в данных полях установим среднее значение <br>
трудового стажа и дохода. Для вычисления стажа выберем среднее значение.

Сперва избавимся от отрицательных значений, т.к. они искозят среднее значение и переведем 'days_employed' в года.

In [ ]:
def true_days_employed(row):
    if row is not None:
        if row < 0:
            return -row/365
        else:
            return row/365
    return row
clients['days_employed'] = clients['days_employed'].apply(true_days_employed)

Проверим остались ли отрицательные значения в трудовом стаже.

In [ ]:
print('Количество строк с отрицательным значением в трудовом стаже =', clients['days_employed'][clients['days_employed'] < 0].count())

Давайте проанализируем стаж более подробней. Для этого посчитаем максимальный и минимальный стаж:

In [ ]:
days_employed_max = clients['days_employed'].max()
days_employed_min = clients['days_employed'].min()
print(f'Минимальный стаж составляет {days_employed_min} лет')
print(f'Максимальный стаж составляет {days_employed_max} лет')

1100 лет стажа. По земному летоисчеслению это нереально. Поэтому для расчета среднего стажа воспользуемся медианой.<br>
Вычислим медиану 'days_employed' сгруппируем данные по возрасту, т.к. стаж из всех категорий больше всего зависит<br>
от возраста. 

NaN и сверхвысокий стаж заменим на медиану. 

Максимально возможный стаж = возраст - 16. Т.к. с 16 лет можно работать.

Проанализируем возраст:

In [ ]:
print('Список самых молодых клиентов')
display(clients['dob_years'].sort_values().head(10))
print('Список самых взрослых клиентов')
display(clients['dob_years'].sort_values().tail(10))
print('Количество клиентов младше 18 лет =', clients['dob_years'][clients['dob_years'] < 19].count())
print('Количество клиентов с возрастом равным 0 =', clients['dob_years'][clients['dob_years'] == 0].count())

Из расчетов видем, что количество клиентов с нулевым значения возраста равно количеству клиентов с возрастом младше 18. Чтобы сформировать группы по возрасту заменим нулевые значения возраста на среднее значения сгруппировав клиентов по типу занятости 'income_type'

In [ ]:
# Сгруппируем значения
median_age = (clients.groupby(['income_type'])).agg({'dob_years':'median'}).rename(columns = {'dob_years': 'median_dob_years'})
# Дабавим столбец с медианой к исходному фрейму
clients = clients.merge(median_age, on = ['income_type'])
# посмотрим сколько пустых строк до замены
print('Пропуски до:', clients['dob_years'][clients['dob_years'] == 0].count())
# заполним пропуски
clients.loc[clients['dob_years'] == 0, 'dob_years'] = clients.loc[clients['dob_years'] == 0, 'median_dob_years']
# пропуски после
print('Пропуски после:', clients['dob_years'][clients['dob_years'] == 0].count())

От нулевых значений в возрасте избавились. Теперь поделим клиентов на четыре группы:
1. От 18 до 30 лет;
2. От 30 до 40 лет;
3. От 40 до 50 лет;
4. 50 лет и больше.

In [ ]:
def kind_of_age(age):
    if age < 31:
        return 1
    elif age < 45:
        return 2
    return 3
clients['kind_of_age'] = clients['dob_years'].apply(kind_of_age)
display(clients.head(10))

Избавимся от пропусков в 'days_employed' исходя из возрастной группы и будем считать медиану со стажем менее максимального возраста.

In [ ]:
# Сгруппируем значения
medians = (clients[clients['days_employed'] < 75].groupby(['kind_of_age'])).agg({'days_employed':'mean'}).rename(columns = {'days_employed': 'median_days_employed'})
# Дабавим столбец с медианой к исходному фрейму
clients = clients.merge(medians, on = ['kind_of_age'])
# посмотрим сколько пустых строк до замены
print('Пропуски до:', clients['days_employed'].isna().sum())
# заполним пропуски
clients.loc[clients['days_employed'].isna(), 'days_employed'] = clients.loc[clients['days_employed'].isna(), 'median_days_employed']

# заменим сверхстаж на медиану

def norm_days_employed(row):
    days_employed = row['days_employed']
    age = row['dob_years']
    median = row['median_days_employed']
    if days_employed >= (age - 16):
        return median
    return days_employed
clients['days_employed'] = clients.apply(norm_days_employed, axis=1)

print('Пропуски после:', clients['days_employed'].isna().sum())

Выполним работу над пропусками в ежемесячном доходе.

Причиной появления пропусков в доходе может служить неправильное внесение данных в таблицу.

Выдвенем гипотезу, что доход зависит от образования, стажа и типа занятости. И проверим так ли это для этого сгруппируем данные по полям 'education_id', 'days_employed', 'income_type' и посмотрим распределение дохода.

In [ ]:
medians_edu = clients.groupby('education_id')['total_income'].median()
print('Зависимость среднего ежемесячного дохода от уровня образования клиента')
display(medians_edu)
medians_emplo = clients.groupby('days_employed')['total_income'].median()
print('Зависимость среднего ежемесячного дохода от стажа клиента')
display(medians_emplo)
medians_inc = clients.groupby('income_type')['total_income'].median()
print('Зависимость среднего ежемесячного дохода от типа занятости клиента')
display(medians_inc)

Из данных видим, что средний ежемесячный доход зависит от типа занятости и от уровня образования клиента. Заменим пропуски на среднее значение дохода 'total_income' сгруппированных по 'income_type' и 'education_id'

In [ ]:
# Сгруппируем значения
median_income = (clients.groupby(['income_type', 'education_id'])).agg({'total_income':'median'}).rename(columns = {'total_income': 'median_total_income'})
# Дабавим столбец с медианой к исходному фрейму
clients = clients.merge(median_income, on = ['income_type', 'education_id'])
# посмотрим сколько пустых строк до замены
print('Пропуски до:', clients['total_income'].isna().sum())
# заполним пропуски
clients.loc[clients['total_income'].isna(), 'total_income'] = clients.loc[clients['total_income'].isna(), 'median_total_income']
# пропуски после
print('Пропуски после:', clients['total_income'].isna().sum())
# удалим лишние столбцы
clients = clients.drop(['median_dob_years', 'median_days_employed', 'median_total_income'], axis=1)

### Вывод

Нашли нулевые значения в столбце возраст 'dob_years' в количестве 101 и заменили на среднее значение возраста.
Нашли нулевые значения в столбце стаж 'days_employed' в количестве 2174 и заменили на среднее значение стажа.
Нашли нулевые значения в столбце стаж 'total_income' в количестве 2174 и заменили на среднее значение дохода.

### Замена типа данных <a class="anchor" id="fourth"></a>

Еще раз взглянем на нашу таблицу:

In [ ]:
display(clients.info())

Значения в стоблце 'days_employed' имеют плавающую точку, но стаж измеряется в годах и дробная часть нас не интересует, поэтому переведем данный столбец к целочисленному значению.

Аналогично с 'dob_years' и 'total_income'. Не считать же нам копейки и дни)

In [ ]:
clients['days_employed'] = clients['days_employed'].astype('int')
clients['dob_years'] = clients['dob_years'].astype('int')
clients['total_income'] = pd.to_numeric(clients['total_income'], errors='coerce')
# проверим новый вывод
display(clients.info())

### Вывод

В столбцах 'days_employed', 'dob_years' и 'total_income' изменили тип данных на целочисленный, теперь приятнее смотреть на данные.

### Обработка дубликатов <a class="anchor" id="fifth"></a>

Проанализируем данные на наличие дубликатов. Для этого к каждому столбцу применим метод .value_counts()

In [ ]:
display(clients['children'].value_counts())

Минус один ребенок конечно не может быть, это что кто-то еще должен ребенка?) 

Здесь явно опечатка и должно быть значение 1.

20 детей - это конечно хорошо, но маловероятно, т.к. были бы люди с 6-19 детьми. Поэтому здесь тоже опечатка и должно быть 2 ребенка.

In [ ]:
# заменим -1 ребенок на 1
clients['children'] = clients['children'].replace(-1, 1)

# заменим 20 детей на 2
clients['children'] = clients['children'].replace(20, 2)

# проверим уникальные значения
display(clients['children'].value_counts())

Теперь значения в столбце 'children' приняли правдивый вид.

Посмотрим на столбец 'education' для этого посмотрим на словарь из 'education_id', 'education' и количество всех строк.

In [ ]:
education_dict = clients.pivot_table(index = ['education_id', 'education'], values = 'debt', aggfunc = ['count'])
education_dict.columns = ['Кол-во заемщиков']
display(education_dict)

Обнаружили различные варианты написания образования. Исправим это приведя значения в столбце 'education' к нижнему регистру

In [ ]:
# приводим к нижнему регистру
clients['education'] = clients['education'].str.lower()

# проверяем результат
education_dict = clients.pivot_table(index = ['education_id', 'education'], values = 'debt', aggfunc = ['count'])
education_dict.columns = ['Кол-во заемщиков']
display(education_dict)

Аналогичные операции выполним с 'family_status' и 'family_status_id'

In [ ]:
family_dict = clients.pivot_table(index = ['family_status_id', 'family_status'], values = 'debt', aggfunc = ['count'])
family_dict.columns = ['Кол-во заемщиков']
display(family_dict)

Дубликатов не обнаружено, но для удобства приведем "Не женат / не замужем" к нижнему регистру

In [ ]:
# приводим к нижнему регистру
clients['family_status'] = clients['family_status'].str.lower()

# проверяем результат
display(clients['family_status'].value_counts())

Проверим на уникальность следующие столбцы 'family_status_id', 'gender', 'income_type', 'debt'

In [ ]:
print('Уникальные значения в "family_status_id"\n')
display(clients['family_status_id'].value_counts())

print('Уникальные значения в "gender"\n')
display(clients['gender'].value_counts())

print('Уникальные значения в "income_type"\n')
display(clients['income_type'].value_counts())

print('Уникальные значения в "debt"\n')
display(clients['debt'].value_counts())

В поле столбца 'gender' обнаружено значение XNA. Отнесем данное поле к большей части группы 'gender'

In [ ]:
clients['gender'] = clients['gender'].replace('XNA', 'F')

# проверим уникальность после замены
clients['gender'].value_counts()

Проверим уникальность полей в столбце 'purpose'

In [ ]:
clients['purpose'].value_counts()

Для работы с этими данными используем лемматизацию и категоризацию в следующих главах.

#### Поиск дубликатов в таблице

In [ ]:
# подсчет дубликатов
clients.duplicated().sum()

Мы обнаружили 71 дубликатов строк. Методом drop_duplicates() удалим их.

In [ ]:
clients = clients.drop_duplicates().reset_index(drop = True)

# проверим количество дубликатов
clients.duplicated().sum()

### Вывод

В исходных данных обнаружили дубликаты и избавились от них в столбцах 'children', 'gender', 'education' и 'family_status'. 
В столбце 'purpose' получили уникальные данные, которые обработаем методом лемматизация и категоризация в следующих главах.

В таблице обнаружили и удалили 71 дубликат.

### Лемматизация <a class="anchor" id="sixth"></a>

Выполним лемматизацию цели получения кредита. Для этого используем библиотеку pymystem и collections. После подсчета будет удобно разбить цель получения кредита на категории.

In [ ]:
# импортируем Mystem для лемматизации полученного списка и Counter для дальнейшего подсчета слов
from pymystem3 import Mystem
from collections import Counter

In [ ]:
# получим список уникальных целей получения кредита
lemmas_purpose = clients['purpose'].unique()

m = Mystem()
big_lemmas = []
for row in lemmas_purpose:
    lemmas = m.lemmatize(row)
    big_lemmas += lemmas

# подсчет упоминаний
display(Counter(big_lemmas))

### Вывод

Выполнили лемматизацию целей получения кредита. Теперь с легкостью выделим категории целей получения кредита.

Наиболее встречающиеся слова:
недвижимость - 10;
автомобиль - 9;
образование - 9;
жилье - 9;
свадьба -3.

### Категоризация данных <a class="anchor" id="seventh"></a>

Из полученной данных после лемматизации разобъем цели получения кредита на категории;
1. Недвижимость + жилье;
2. Автомобиль;
3. Образование;
4. Свадьба.

Теперь добавим категорию цели получения кредита:

In [ ]:
# функция принимает строку и возвращает категорию
def purpose_category(row):
    purpose = row['purpose']
    purpose = m.lemmatize(purpose)
    if 'свадьба' in purpose:
        return 'свадьба'
    elif 'автомобиль' in purpose:
        return 'автомобиль'
    elif 'образование' in purpose:
        return 'образование'
    return 'недвижимость'

# добавим новый столбец 'purpose_category' к таблице
clients['purpose_category'] = clients.apply(purpose_category, axis=1)

# выведем таблицу и посмотрим на результат
clients.head()

Выведем словарь по цели получения кредита.

In [ ]:
purpose_dict = clients.pivot_table(index = ['purpose_category', 'purpose'], values = 'debt', aggfunc = ['count'])
purpose_dict.columns = ['Кол-во заемщиков']
display(purpose_dict)

Посчитаем категорию зарплат. Для этого посчитаем медиану, минимум, максимум и медианы между медианой и минимумом и между медианой и максимумом.

In [ ]:
# посчитаем среднее значение для дохода, а также минимум и максимум
income_median = clients['total_income'].median()
income_min = clients['total_income'].min()
income_max = clients['total_income'].max()

# посчитаем среднее значение между медианой и нижней границей дохода
income_group_1_median = clients['total_income'].quantile(0.25)

# посчитаем среднее значение между медианой и верхней границей дохода
income_group_2_median = clients['total_income'].quantile(0.75)

# функция принимает строку и возвращает категорию
def income_category(row):
    income = row['total_income']
    if income <= income_group_1_median:
        return 1
    elif income <= income_median:
        return 2
    elif income <= income_group_2_median:
        return 3
    return 4

# добавим новый столбец 'income_category' к таблице
clients['income_category'] = clients.apply(income_category, axis=1)
print('Первая категория имеет доход от {:.0f} до {:.0f} включительно'.format(income_min, income_group_1_median))
print('Вторая категория имеет доход от {:.0f} до {:.0f} включительно'.format(income_group_1_median, income_median))
print('Третья категория имеет доход от {:.0f} до {:.0f} включительно'.format(income_median, income_group_2_median))
print('Четвертая категория имеет доход от {:.0f} до {:.0f} включительно'.format(income_group_2_median, income_max)) 

### Вывод

Благодаря лемматизации мы разбили цели получения кредита на 4 категории: 
1. Недвижимость + жилье;
2. Автомобиль;
3. Образование;
4. Свадьба.

И добавили категорию к таблице.

Разбили уровень дохода на категории:
1. Первая категория имеет доход от 20 667 до 107 516 включительно
2. Вторая категория имеет доход от 107 516 до 143 708 включительно
3. Третья категория имеет доход от 143 708 до 198 307 включительно
4. Четвертая категория имеет доход от 198 307 до 2 265 604 включительно

## Шаг 3. Ответьте на вопросы <a class="anchor" id="eighth"></a>

### - Есть ли зависимость между наличием детей и возвратом кредита в срок? <a class="anchor" id="ninth"></a>

Вычислим зависимость между наличием детей и возвратом кредита в срок, для этого поделим сумму задолженностей по возврату кредитов на общее количество кредитов сгруппированных по столбцу 'children'

In [ ]:
children_debt = clients.pivot_table(index = ['children'], values = 'debt', aggfunc = ['mean', sum, 'count'])
children_debt.columns = ['% невозврата', 'Кол-во должников', 'Кол-во заемщиков']
display(children_debt.style.format({'% невозврата':'{:.2%}'}))

### Вывод

Клиенты без детей чаще остальных возвращают кредиты в срок. 

Клиенты с 5 детьми хоть и всегда возвращают кредиты в срок, но выборка слишком мала (9 клиентов).

### - Есть ли зависимость между семейным положением и возвратом кредита в срок? <a class="anchor" id="tenth">

Вычислим зависимость между семейным положением и возвратом кредита в срок, для этого поделим сумму задолженностей по возврату кредитов на общее количество кредитов сгруппированных по столбцу 'family_status'

In [ ]:
family_debt = clients.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['mean', sum, 'count'])
family_debt.columns = ['% невозврата', 'Кол-во должников', 'Кол-во заемщиков']
display(family_debt.style.format({'% невозврата':'{:.2%}'}))

### Вывод

Чаще всего возвращают кредиты люди в разводе. Реже остальных не женатые или не замужние клиенты. Клиенты, которые пошли на такой ответственный шаг как замужество и женитьба, более ответственно относятся и к возврату кредита.

### - Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a class="anchor" id="eleventh">

Вычислим зависимость между уровнем дохода и возвратом кредита в срок, для этого поделим сумму задолженностей по возврату кредитов на общее количество кредитов сгруппированных по столбцу 'total_income'

In [ ]:
income_debt = clients.pivot_table(index = ['income_category'], values = 'debt', aggfunc = ['mean', sum, 'count'])
income_debt.columns = ['% невозврата', 'Кол-во должников', 'Кол-во заемщиков']
display(income_debt.style.format({'% невозврата':'{:.2%}'}))

### Вывод

Клиенты с высоким уровнем дохода чаще возвращают кредит.

### - Как разные цели кредита влияют на его возврат в срок? <a class="anchor" id="twelfth">

Вычислим зависимость между различными целями кредита и возвратом кредита в срок, для этого поделим сумму задолженностей по возврату кредитов на общее количество кредитов сгруппированных по столбцу 'purpose_category'

In [ ]:
purpose_debt = clients.pivot_table(index = ['purpose_category'], values = 'debt', aggfunc = ['mean', sum, 'count'])
purpose_debt.columns = ['% невозврата', 'Кол-во должников', 'Кол-во заемщиков']
display(purpose_debt.style.format({'% невозврата':'{:.2%}'}))

### Вывод

Кредит на покупку недвижимости возвращается чаще всего, а вот на покупку автомобиля или образования реже всего.

Зависимость возврата кредита в срок от наличия детей и семейного положения вычислим используя метод .pivot_table()

In [ ]:
family_children = clients.pivot_table(index = ['family_status', 'children'], values = 'debt', aggfunc = ['mean', sum, 'count'])
family_children.columns = ['% невозврата', 'Кол-во должников', 'Кол-во заемщиков']
display(family_children.style.format({'% невозврата':'{:.2%}'}))

## Шаг 4. Общий вывод <a class="anchor" id="thirteenth">

Краткий обзор проведенной работы.

Для исследования надежности заемщиков мы открыли файл Заказчика и изучили информацию. Статистика о платежеспособности клиентов состоит из таблицы в 12 столбцов и 21 525 строк. Обнаружили артефакты в виде отрицательного трудового стажа, трудового стажа более 1 000 лет и нулевого возраста.

Отработали пропущенные значения в трудовом стаже и в ежемесячном доходе в количестве 2 174, заменив их средним значением.

Изменили тип данных значений в столбцах общий трудовой стаж, возраст клиента в годах и ежемесячный доход на целочисленный.

Обнаружили дубликаты значений в столбцах количество детей в семье, пол клиента, образование и семейное положение и избавились от них в столбцах.

Используя лемматизации мы разбили цели получения кредита на категории: 
1. Недвижимость;
2. Автомобиль;
3. Образование;
4. Свадьба.

Разбили уровень дохода на категории:
1. Первая категория имеет доход от 20667 до 107516 включительно;
2. Вторая категория имеет доход от 107516 до 143708 включительно;
3. Третья категория имеет доход от 143708 до 198307 включительно;
4. Четвертая категория имеет доход от 198307 до 2265604 включительно.

Установили зависимость между возвратом кредита в срок и между:
- наличием детей;
- уровнем дохода;
- семейным положением;
- целью кредита.

Главные выводы:

Семейное положение и количество детей клиента влияет на факт погашения кредита в срок.

Самые рискованные группы:
1. Не женатые/не замужние клиенты;
2. Клиенты в гражданском браке с детьми.

Наименее рискованные группы - это клиенты без детей.

Рекомендации. 

Для прогнозирования советуем отобрать следующие признаки:
1. Полеты за границу за последний 12 месяцев;
2. Наличие / отсутствие автомобиля;
3. Наличие / отсутствие недвижимости.

Во избежание появления дубликатов предлагаем разработать скрипт, который разбивал бы введеные значения по категориям, либо добавить возможность выбирать значения из списка.

Для избежания появления пропущенных значений предлагаем привести поля к единому формату.

### Чек-лист готовности проекта

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.